## 형태소 분석

형태소 분석(Morphoogical Analysis)는 문장을 *형태소* 라는 의미를 갖는 최소 단위로 분리하고 품사를 판별하는 작업이다. 형태소 분석은 기계번역, 텍스트 마이닝 등의 분야에서 사용한다.

영어문장의 형태소는 형태소 마다 띄어 쓰기를 하기 때문에 쉬울 수 있다. 예외소 *It's, don't* 등이 있는데 이것은 특정 단어를 *it is, do not* 으로 변환하므로 큰 문제가 되지 않는다.

아시아 계열의 문장은 노력이 필요하다. 대표적으로 문법 규칙에 의한 방법과 확률적 언어 모델을 사용하는 방법이 있다. 최근 확률적 언어 모델을 사용한 형태소 분석이 정밀도가 높고 많이 나오고 있다. 이 두 가지 모두 품사 사전과 문법 사전을 기반으로 대조해서 형태소를 분석한다.

## N-gram 으로 유사도 구하기

N-gram이란 텍스트에서 *이웃한 n개의 문자*를 의미한다. 서로 다른 2 문장을 N-gram으로 비교해 보면 출현하는 단어의 종류와 빈도를 확이할 수 있다. 이를 활용하면 논문 도용 등을 확인할 수 있다. 또한 코드의 라이센스 확인에도 활용할 수 있다.

다음 2 문장은 어느 정도 유사할까?

- A. 오늘 강남에서 맛있는 스파게티를 먹었다.
- B. 강남에서 먹었던 오늘의 스파게티는 맛있었다.

ngram() 함수를 정의해 보자

In [1]:
def ngram(s, num):
    res = []
    slen = len(s) - num + 1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

def diff_ngram(s1, s2, num):
    a = ngram(s1, num)
    b = ngram(s2, num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i == j:
                cnt +=1
                r.append(i)
    return cnt / len(a), r

In [3]:
a = '오늘 강남에서 맛있는 스파게티를 먹었다.'
b = '강남에서 먹었던 오늘의 스파게티는 맛있었다.'

다음은 2글자씩 끊어 읽는다는 의미로 2-gram 이라고 한다.

In [4]:
ngram(a, 2)

['오늘',
 '늘 ',
 ' 강',
 '강남',
 '남에',
 '에서',
 '서 ',
 ' 맛',
 '맛있',
 '있는',
 '는 ',
 ' 스',
 '스파',
 '파게',
 '게티',
 '티를',
 '를 ',
 ' 먹',
 '먹었',
 '었다',
 '다.']

2-gram으로 비교하면?

In [5]:
r2, word2 = diff_ngram(a, b, 2)
print('2-gram:', r2, word2)

2-gram: 0.7619047619047619 ['오늘', '강남', '남에', '에서', '서 ', ' 맛', '맛있', '는 ', ' 스', '스파', '파게', '게티', ' 먹', '먹었', '었다', '다.']


In [6]:
r3, word3 = diff_ngram(a, b, 3)
print('3-gram:', r3, word3)

3-gram: 0.45 ['강남에', '남에서', '에서 ', ' 맛있', ' 스파', '스파게', '파게티', ' 먹었', '었다.']


다음 문장은? 2그람으로 비교하면 74% 정도로, 문장 내용을 순서만 바꾼 것으로는 유사도가 높게 나온다.

In [7]:
a = '머신러닝은 매우 재미있는 기술이라 공부하고 있습니다'
b = '공부하면 재미있는 기술이라 머신러닝을 배우고 있습니다'

In [8]:
r2, word2 = diff_ngram(a, b, 2)
print('2-gram: ', r2, word2)

2-gram:  0.7407407407407407 ['머신', '신러', '러닝', ' 재', '재미', '미있', '있는', '는 ', ' 기', '기술', '술이', '이라', '라 ', '공부', '부하', '고 ', ' 있', '있습', '습니', '니다']


In [9]:
r3, word3 = diff_ngram(a, b, 3)
print('3-gram: ', r3, word3)

3-gram:  0.6153846153846154 ['머신러', '신러닝', ' 재미', '재미있', '미있는', '있는 ', '는 기', ' 기술', '기술이', '술이라', '이라 ', '공부하', '고 있', ' 있습', '있습니', '습니다']


In [10]:
# 비슷한 문장으로 2그람 유사도를 확인해보자
a = '본문과 관계 없는 내용이지만 마시멜로는 맛있습니다'
b = '마시멜로는 본문과 전혀 관계 없이 맛있습니다'

r2, word2 = diff_ngram(a, b, 2)
print('2-gram: ', r2, word2)

2-gram:  0.6923076923076923 ['본문', '문과', '과 ', ' 관', '관계', '계 ', ' 없', '는 ', '마시', '시멜', '멜로', '로는', '는 ', ' 맛', '맛있', '있습', '습니', '니다']


In [11]:
# 관계없는 문장으로 2그람 유사도를 확인해보자
a = '파이썬 프로그래밍에서 중요한 것은 블록입니다'
b = '마시멜로는 본문과 전혀 관계 없이 맛있습니다'

r2, word2 = diff_ngram(a, b, 2)
print('2-gram: ', r2, word2)

2-gram:  0.043478260869565216 ['니다']


N-gram을 이용하면 쉽게 문장 유사도를 확인할 수 있다. 이런 특징으로 문장 도용을 확인할 수 있고, 검색 엔진 등에서 웹 문서 유사도를 확인할 수 있는 다양성이 있다.

---

## 한국어 형태소 분석 라이브러리

여러 형태소 분석 오픈소스 라이브러리가 있는ㄷ여기서는 KoNLPy(http://konlpy.org/ko/latest)를 이용하겠다. 이 KoNLPy를 사용하면 *한나눔*,*꼬꼬마*,*Komoran*,*MeCab*, *트위터* 등의 형태소 분석기를 이용할 수 있다.

### 설치

konlpy 를 설치한다.

```
pip install konlpy --user
```

In [1]:
!pip install konlpy

형태소 tag 클래스 밑에 형태소 분석기 클래스가 있다.
 - Hannanum class
 - Kkma class
 - Komoran class
 - Mecab class
 - Okt clas

간단한 형태소 분석을 해보자, 

### Okt class

Open Korean Text [Okt class](http://konlpy.org/ko/latest/api/konlpy.tag/#okt-class) 는 Scala로 작성되서 한글 토큰나이저이다.
> v0.5 이전에 Twitter class라고 했다

```
class konlpy.tag._okt.Okt(jvmpath=None, max_heap_size=1024)
```


In [2]:
from konlpy.tag import Okt
okt = Okt()
malist = okt.pos("아버지 가방에 들어 가신다", norm=True, stem=True)
malist

[('아버지', 'Noun'),
 ('가방', 'Noun'),
 ('에', 'Josa'),
 ('들다', 'Verb'),
 ('가다', 'Verb')]

POS tagger. In contrast to other classes in this subpackage, this POS tagger doesn't have a flatten option, but has norm and stem options. Check the parameter list below.

```
pos(phrase, norm=False, stem=False, join=False)
 - norm -- If True, normalize tokens.
 - stem -- If True, stem tokens.
 - join -- If True, returns joined sets of morph and tag.

```

`norm=True` 를 주면 *그래욬 ㅋㅋ?* 같은 단어를 *그래요* 로 정정해서 반환해 주고, stem 을 사용하면 *그렇다*라는 원형을 찾아 준다.

KONLPy에 형태소 분석기 중 *아버지 가방에 들어가신다* 를 제대로 분석하는 것은 Kkma와 Okt 가 가능하다. Okt는 품사를 구분할 수 있고, Kkma는 품사가 NNG, JKM, VV, EPH, EFN 같이 나온다.

## 출현 빈도 분석

[KoNLPy는 말뭉치 파일(Corpus File)](http://konlpy.org/ko/latest/references/#corpora) 1) kolay: 한국 법률 말뭉치, 2) kobill: 국회의안 말뭉치을 제공하고 있다.

 - http://konlpy.org/ko/latest/references/#corpora

> [말뭉치를 이용한 관용구 사전의 의미 정보
> (http://web.yonsei.ac.kr/bk21yskor/board/12/20070220194945637_권경일07.pdf)


In [5]:
from konlpy.corpus import kolaw
kolaw = kolaw.open('constitution.txt').read()
kolaw[:100]

'대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의'

In [6]:
from konlpy.tag import Okt

okt = Okt()
word_dic = {}
lines = kolaw.split('\n')
for line in lines:
    malist = okt.pos(line)
    for word in malist:
        if word[1] == 'Noun':
            if not (word[0] in word_dic):
                word_dic[word[0]] = 0
            word_dic[word[0]] += 1
#빈도
keys = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
for word, count in keys[:50]:
    print("{0}({1}) ".format(word, count), end="")
print()

제(175) 법률(127) 정(89) 수(88) 대통령(83) 국가(73) 국회(68) 국민(61) 관(58) 때(55) 헌법(53) 그(47) 이(38) 모든(37) 바(37) 위(36) 기타(26) 및(25) 사항(23) 권리(21) 의원(21) 안(21) 정부(20) 선거(20) 자유(20) 임명(20) 직무(19) 국무총리(19) 경제(18) 조직(18) 의무(18) 국회의원(18) 임기(18) 경우(17) 이상(17) 공무원(17) 국무위원(17) 대법원(17) 의결(17) 범위(16) 장(15) 법관(15) 재적(14) 보호(14) 관리(14) 항의(14) 헌법재판소(14) 정책(14) 회의(14) 정당(14) 


---

# Word2Vec으로 문장을 벡터로 변환하기

분리한 단어들 사이의 연관을 분석하기 위해서는 단어와 단어를 벡터로 전환해 유사도를 분석해야 한다.

    6-2-1. Word2Vec
    6-2-2. Gensim 설치
    6-2-3. Gensim의 Word2Vec으로 "토지"를 읽어보기
    6-2-4. 위키피디아 한국어 버전을 사전으로 사용해보기
    6-2-5. 위키피디아 데이터로 놀아보기


## Word2Vec

Word2Vec은 문자 내부 단어를 벡터로 변환해준다. 단여 연결을 기반으로 연관성을 벡터로 만들어 준다. 벡터를 사용해 단어의 의미를 파악할 수 있다.

Word2Vec을 구현하는 도구는 많은데 파이썬으로 실행할 수 있는 Gensim 을 사용해 보자, Gensim은 자연어를 처리하는 라이브러리이다. 이 중에 Word2vec 기능이 있다.

 - https://radimrehurek.com/gensim/
 
설치는

```
pip install --upgrade gensim --user
```


In [7]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 1.2 MB/s eta 0:00:01    |████▋                           | 3.5 MB 741 kB/s eta 0:00:28
     |████████████████████████████████| 113 kB 2.5 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=6e44a69eba3111873ec753ff7ca7b918fb788e4fca561cfec6ec2e3acb9f6056
  Stored in directory: /Users/qkboo/Library/Caches/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open


## Gensim의 Word2Vec으로 읽어보기



In [8]:
from konlpy.tag import Okt
from konlpy.corpus import kolaw
from gensim.models import word2vec

kolaw = kolaw.open('constitution.txt').read()

In [9]:
okt = Okt()
results = []
lines = kolaw.split('\n')
for line in lines:
    #형태소분석: 단어기본형 사용.
    malist = okt.pos(line, norm=True, stem=True)
    r = []
    for word in malist:
        #어미/조사/구두점 등은 대상에서 제외
        if not word[1] in ['Josa', 'Eomi', 'Punctuation']:
            r.append(word[0])
    rl = (" ".join(r)).strip()
    results.append(rl)
    print(rl)
#파일로 출력하기
wakati_file = 'data/kolaw.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(results))
# word2vec 모델
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, size=200, window=10, hs=1, min_count=2, sg=1)
model.save('data/kolaw.model')
print('OK')

대한민국 헌법

유구 역사 전통 빛나다 우리 대 한 국민 3 1 운동 건립 되다 대한민국 임시정부 법 통과 불의 항거 4 19 민주 이념 계승 조국 민주 개혁 평화 적 통일 사명 입 각하 정의 인도 동포 애 로써 민족 단결 공고 히 하다 모든 사회 적 폐습 불의 타파 하다 자율 조화 바탕 자유민주 적 기 본 질서 더욱 확고하다 하다 정치 경제 사회 문화 모든 영역 있다 각인 기회 균등하다 하다 능력 최고 도로 발휘 하다 하다 자유 권리 따르다 책임 의무 완수 하다 하다 안 국민 생활 균등하다 향상 기하 밖 항구 적 세계 평화 인류 공영 이바지 함 우리 들 우리 들 자손 안전 자유 행복 영원하다 확보 하다 것 다짐 하다 1948년 7월 12일 에 제정 되다 8 차 걸치다 개정 되다 헌법 이제 국회 의결 거치다 국민투표 의하다 개정 하다

제 1 장 총 강
제 1조 ① 대한민국 민주공화국
② 대한민국 주권 국민 있다 모든 권력 국민 나오다
제 2조 ① 대한민국 국민 되다 요건 법률 정 하다
② 국가 법률 정 하다 바 의하다 재외국민 보호 하다 의무 지다
제 3조 대한민국 영토 한반도 그 부속 도서 하다
제 4조 대한민국 통일 지향 하다 자유민주 적 기 본 질서 입각 평화 적 통일 정책 수립 이르다 추진 다
제 5조 ① 대한민국 국제 평화 유지 노력 침략 적 전쟁 부인 하다
② 국군 국가 안전보장 국토 방위 신성하다 의무 수행 함 사명 하다 그 정치 적 중립성 준수 되다
제 6조 ① 헌법 의하다 체결 공포 되다 조약 일반 적 승인 되다 국제 법규 국내법 같다 효력 가지다
② 외국인 국제 법 조약 정 하다 바 의하다 그 지위 보장 되다
제 7조 ① 공무원 국민 전체 대한 봉사자 국민 대하 책임 지다
② 공무원 신분 정치 적 중립성 법률 정 하다 바 의하다 보장 되다
제 8조 ① 정당 설립 자유 복수정당제 보장 되다
② 정당 그 목적 조직 활동 민주 적다 하다 국민 정치 적 의사 형성 참여 하다 필요하다 조직 가다 하다
③ 정당 법률 정 하다 바 의하다 국가 보호 받다

kolwa.model 로 저장한 모델을 사용하자

In [13]:
model = word2vec.Word2Vec.load('data/kolaw.model')

유사한 단어를 확인할 때 wv.most_similar()를 사용한다.

In [12]:
result = model.wv.most_similar(positive=['국민'])
result

[('의무', 0.9963914155960083),
 ('모든', 0.9957342743873596),
 ('정', 0.995196521282196),
 ('개발', 0.9951783418655396),
 ('보호', 0.9951227903366089),
 ('직무', 0.9939312934875488),
 ('필요하다', 0.9937732219696045),
 ('및', 0.9935662746429443),
 ('사항', 0.9934930205345154),
 ('급', 0.9931744933128357)]